In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
# To check whether the pipeline is running initial load or incremental load
# dbutils.widgets.text("incremental_flag", "0")
# dbutils.widgets.get("incremental_flag")

In [0]:
# incremental_flag = dbutils.widgets.get("incremental_flag")
# incremental_flag

In [0]:
# Connecting to Azure Data Lake 
spark.conf.set('fs.azure.account.key.datalakeazuredatabricks.dfs.core.windows.net', 'LuUWQy2pFP32+FiO94ok2cwkoyLWOI2zO2kFBtsCHBqt4t/ohW8squAMNX7wPMEzMebxoaHk3k5o+AStROC22A==')

In [0]:
# Customer Dimension
df_customer = spark.sql("""
                        select * from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Customer`
                        """)

df_customer.display()

In [0]:
# Product Dimension
df_product = spark.sql("""
                        select * from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Product`
                        """)

df_product.display()

In [0]:
# Date Dimension
df_date = spark.sql("""
                        select * from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Date`
                        """)

df_date.display()

In [0]:
# SalesRegion Dimension
df_region = spark.sql("""
                        select * from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Region`
                        """)

df_region.display()

In [0]:
# Load Silver Table also
df_silver = spark.sql("""
                      select * from parquet.`abfss://silver@datalakeazuredatabricks.dfs.core.windows.net`
                      """)
df_silver.display()

In [0]:
# Create Fact table
fact_df = df_silver.join(df_customer, df_silver['CustomerID'] == df_customer['CustomerID'], 'left').join(df_product, df_silver['ProductID'] == df_product['ProductID'], 'left').join(df_date, df_silver['OrderDate'] == df_date['OrderDate'], 'left').join(df_region, df_silver['SalesRegion'] == df_region['SalesRegion'], 'left').select(df_customer['Dim_Customer_Key'], df_product['Dim_Product_Key'], df_date['Dim_Date_Key'], df_region['Dim_Region_Key'], df_silver['UnitPrice'], df_silver['TotalPrice'], df_silver['Quantity'])

fact_df.display()

In [0]:
# SCD Type1 = Upsert or Update + Insert
table_name = 'gold.dim_factsales'
path = 'abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_factsales'

if DeltaTable.isDeltaTable(spark, path):
    deltaTable = DeltaTable.forPath(spark, path)
    deltaTable.alias('target').merge(fact_df.alias('source'),'target.Dim_Customer_key==source.Dim_Customer_Key and target.Dim_Product_Key==source.Dim_Product_Key and target.Dim_Date_Key==source.Dim_Date_Key and target.Dim_Region_Key==source.Dim_Region_Key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    print('Good Working')
else:
    fact_df.write.format('delta').mode('overwrite').option('mergeSchema', 'true').save(path)
    print('Table is created')

fact_df.display()
